## House Hunting in SF: Crime-based clustering

In [5]:
import pandas as pd
import requests


import numpy as np # library to handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')



Libraries imported.


### Import Redfin housing data

In [6]:
# The code was removed by Watson Studio for sharing.

,ZIP OR POSTAL CODE,ADDRESS,LATITUDE,LONGITUDE
0,94132,257 N Lake Merced Hls,37.712635,-122.481353
1,94112,1957 San Jose Ave,37.725763,-122.442057
2,94134,1328-1330 Silver Ave,37.731009,-122.409813
3,94123,1501 Greenwich St #202,37.800064,-122.424680
4,94118,69 Palm Ave #4,37.784415,-122.458306


In [3]:
address = 'San Francisco'

geolocator = Nominatim(user_agent="sf_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of SF are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of SF are 37.7790262, -122.4199061.


In [4]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
# create map of SF using latitude and longitude values
map_sf = folium.Map(location = [latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, address in zip(sf_zip['LATITUDE'], sf_zip['LONGITUDE'], sf_zip['ADDRESS']):
    label = '{}'.format(address)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.5,
        parse_html = False).add_to(map_sf)  
    
map_sf

Solving environment: done

# All requested packages already installed.



In [7]:

body = client_9c1de0070d30482d86f53bc9daa88cdf.get_object(Bucket='courseradatasciencecapstone-donotdelete-pr-jwdwzjfc8yorsr',Key='Crime_data_feb_2020_clean.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

sf_crimes = pd.read_csv(body)
sf_crimes.head()


,ADDRESS,ADDRESS Latitude,ADDRESS Longitude,CRIME,CRIME Latitude,CRIME Longitude,CRIME Category
0,257 N Lake Merced Hls,37.712635,-122.481353,Assault,37.716375,-122.479185,Simple Assault
1,257 N Lake Merced Hls,37.712635,-122.481353,Malicious Mischief,37.716375,-122.479185,Vandalism
2,257 N Lake Merced Hls,37.712635,-122.481353,Assault,37.716246,-122.483275,Simple Assault
3,1957 San Jose Ave,37.725763,-122.442057,Burglary,37.727861,-122.438085,Burglary - Hot Prowl
4,1957 San Jose Ave,37.725763,-122.442057,Drug Offense,37.723130,-122.435977,Drug Violation


In [8]:
sf_crimes.groupby('ADDRESS').count()

,ADDRESS Latitude,ADDRESS Longitude,CRIME,CRIME Latitude,CRIME Longitude,CRIME Category
ADDRESS,,,,,,
10 Innes Ct #203,22,22,22,22,22,22
10 Innes Ct #303,44,44,44,44,44,44
10 Innes Ct #306,22,22,22,22,22,22
1001 17th St #403,62,62,62,62,62,62
1029 Natoma St #2,168,168,168,168,168,168
1133 Russia Ave,11,11,11,11,11,11
1139 Judah St,37,37,37,37,37,37
1160 Mission St #1907,346,346,346,346,346,346
1182 Palou Ave,44,44,44,44,44,44


In [9]:
print('There are {} uniques categories.'.format(len(sf_crimes['CRIME Category'].unique())))

There are 45 uniques categories.


### Analyze each Neighborhood

In [10]:
# one hot encoding
sf_onehot = pd.get_dummies(sf_crimes[['CRIME Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
sf_onehot['ADDRESS'] = sf_crimes['ADDRESS'] 
# move neighborhood column to the first column
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]
sf_onehot.head()

,ADDRESS,Aggravated Assault,Arson,Burglary - Commercial,Burglary - Hot Prowl,Burglary - Other,Burglary - Residential,Child Abuse,Drug Violation,Embezzlement,Family Offenses,Fire Report,Forgery And Counterfeiting,Fraud,Larceny - Auto Parts,Larceny - From Vehicle,Larceny Theft - Bicycle,Larceny Theft - From Building,Larceny Theft - Other,Larceny Theft - Pickpocket,Larceny Theft - Purse Snatch,Larceny Theft - Shoplifting,Liquor Law Violation,Missing Adult,Missing Person,Motor Vehicle Theft,Motor Vehicle Theft (Attempted),Other,Prostitution,Recovered Vehicle,Robbery - Carjacking,Robbery - Commercial,Robbery - Other,Robbery - Street,Sex Offense,Simple Assault,Stalking,Stolen Property,Suicide,Theft From Vehicle,Vandalism,Vehicle Impounded,Vehicle Misplaced,Warrant,Weapons Offense
0,257 N Lake Merced Hls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,257 N Lake Merced Hls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,257 N Lake Merced Hls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,1957 San Jose Ave,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1957 San Jose Ave,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
## group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

sf_grouped = sf_onehot.groupby('ADDRESS').mean().reset_index()
sf_grouped

,ADDRESS,Aggravated Assault,Arson,Burglary - Commercial,Burglary - Hot Prowl,Burglary - Other,Burglary - Residential,Child Abuse,Drug Violation,Embezzlement,Family Offenses,Fire Report,Forgery And Counterfeiting,Fraud,Larceny - Auto Parts,Larceny - From Vehicle,Larceny Theft - Bicycle,Larceny Theft - From Building,Larceny Theft - Other,Larceny Theft - Pickpocket,Larceny Theft - Purse Snatch,Larceny Theft - Shoplifting,Liquor Law Violation,Missing Adult,Missing Person,Motor Vehicle Theft,Motor Vehicle Theft (Attempted),Other,Prostitution,Recovered Vehicle,Robbery - Carjacking,Robbery - Commercial,Robbery - Other,Robbery - Street,Sex Offense,Simple Assault,Stalking,Stolen Property,Suicide,Theft From Vehicle,Vandalism,Vehicle Impounded,Vehicle Misplaced,Warrant,Weapons Offense
0,10 Innes Ct #203,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.318182,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.090909,0.000000,0.045455,0.000000,0.000000,0.045455,0.045455,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.136364,0.000000,0.000000,0.000000,0.000000
1,10 Innes Ct #303,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.318182,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.090909,0.000000,0.045455,0.000000,0.000000,0.045455,0.045455,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.136364,0.000000,0.000000,0.000000,0.000000
2,10 Innes Ct #306,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.318182,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.090909,0.000000,0.045455,0.000000,0.000000,0.045455,0.045455,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.136364,0.000000,0.000000,0.000000,0.000000
3,1001 17th St #403,0.000000,0.000000,0.016129,0.000000,0.032258,0.000000,0.000000,0.000000,0.00000,0.016129,0.000000,0.000000,0.000000,0.000000,0.709677,0.000000,0.000000,0.048387,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.080645,0.000000,0.000000,0.032258,0.000000
4,1029 Natoma St #2,0.017857,0.005952,0.035714,0.000000,0.035714,0.011905,0.000000,0.053571,0.00000,0.005952,0.000000,0.000000,0.011905,0.000000,0.250000,0.017857,0.059524,0.047619,0.011905,0.000000,0.029762,0.000000,0.053571,0.011905,0.041667,0.000000,0.053571,0.000000,0.011905,0.000000,0.011905,0.011905,0.005952,0.000000,0.071429,0.000000,0.005952,0.000000,0.017857,0.083333,0.005952,0.000000,0.005952,0.011905
5,1133 Russia Ave,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.181818,0.000000,0.000000,0.272727,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.181818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,1139 Judah St,0.000000,0.000000,0.000000,0.000000,0.000000,0.108108,0.000000,0.000000,0.00000,0.027027,0.000000,0.000000,0.000000,0.000000,0.297297,0.000000,0.027027,0.162162,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.054054,0.000000,0.000000,0.027027,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.054054,0.162162,0.000000,0.000000,0.000000,0.000000
7,1160 Mission St #1907,0.040462,0.005780,0.008671,0.002890,0.020231,0.005780,0.000000,0.170520,0.00289,0.008671,0.005780,0.002890,0.005780,0.000000,0.083815,0.005780,0.026012,0.104046,0.017341,0.000000,0.011561,0.000000,0.037572,0.011561,0.023121,0.000000,0.086705,0.000000,0.028902,0.000000,0.002890,0.043353,0.005780

In [12]:
# print each neighborhood along with the top 5 most common crimes

num_top_crimes = 5
for hood in sf_grouped['ADDRESS']:
    print("----"+hood+"----")
    temp = sf_grouped[sf_grouped['ADDRESS'] == hood].T.reset_index()
    temp.columns = ['CRIME','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_crimes))
    print('\n')

----10 Innes Ct #203----
                    CRIME  freq
0  Larceny - From Vehicle  0.32
1               Vandalism  0.14
2                   Other  0.09
3      Aggravated Assault  0.05
4          Simple Assault  0.05


----10 Innes Ct #303----
                    CRIME  freq
0  Larceny - From Vehicle  0.32
1               Vandalism  0.14
2                   Other  0.09
3      Aggravated Assault  0.05
4          Simple Assault  0.05


----10 Innes Ct #306----
                    CRIME  freq
0  Larceny - From Vehicle  0.32
1               Vandalism  0.14
2                   Other  0.09
3      Aggravated Assault  0.05
4          Simple Assault  0.05


----1001 17th St #403----
                    CRIME  freq
0  Larceny - From Vehicle  0.71
1               Vandalism  0.08
2   Larceny Theft - Other  0.05
3     Motor Vehicle Theft  0.03
4                 Warrant  0.03


----1029 Natoma St #2----
                           CRIME  freq
0         Larceny - From Vehicle  0.25
1                  

In [13]:
# put into dataframe
def return_most_common_crimes(row, num_top_crimes):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_crimes]

In [14]:
num_top_crimes = 5
indicators = ['st', 'nd', 'rd']
# create columns according to number of top crimes
columns = ['ADDRESS']
for ind in np.arange(num_top_crimes):
    try:
        columns.append('{}{} Most Common Crime'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Crime'.format(ind+1))
# create a new dataframe
neighborhoods_crimes_sorted = pd.DataFrame(columns=columns)
neighborhoods_crimes_sorted['ADDRESS'] = sf_grouped['ADDRESS']
for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_crimes_sorted.iloc[ind, 1:] = return_most_common_crimes(sf_grouped.iloc[ind, :], num_top_crimes)
neighborhoods_crimes_sorted.shape

(60, 6)

## Cluster!

In [15]:
# set number of clusters
kclusters = 6
sf_grouped_clustering = sf_grouped.drop('ADDRESS', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 0, 1, 0, 1, 1, 0], dtype=int32)

In [16]:
# add clustering labels

neighborhoods_crimes_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sf_merged = sf_zip
sf_merged = sf_merged.merge(neighborhoods_crimes_sorted, on = 'ADDRESS')

sf_merged.head()

,ZIP OR POSTAL CODE,ADDRESS,LATITUDE,LONGITUDE,Cluster Labels,1st Most Common Crime,2nd Most Common Crime,3rd Most Common Crime,4th Most Common Crime,5th Most Common Crime
0,94132,257 N Lake Merced Hls,37.712635,-122.481353,4,Simple Assault,Vandalism,Weapons Offense,Fire Report,Larceny Theft - Pickpocket
1,94112,1957 San Jose Ave,37.725763,-122.442057,0,Larceny - From Vehicle,Motor Vehicle Theft,Vandalism,Missing Person,Weapons Offense
2,94134,1328-1330 Silver Ave,37.731009,-122.409813,1,Motor Vehicle Theft,Larceny Theft - Other,Larceny - From Vehicle,Vandalism,Robbery - Other
3,94123,1501 Greenwich St #202,37.800064,-122.424680,0,Larceny - From Vehicle,Vandalism,Larceny Theft - Other,Motor Vehicle Theft,Burglary - Other
4,94118,69 Palm Ave #4,37.784415,-122.458306,0,Larceny - From Vehicle,Larceny Theft - Other,Vandalism,Theft From Vehicle,Robbery - Commercial


In [15]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['LATITUDE'], sf_merged['LONGITUDE'], sf_merged['ADDRESS'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.2).add_to(map_clusters)
       
map_clusters

### Cluster 0

In [17]:
sf_merged.loc[sf_merged['Cluster Labels'] == 0, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Crime,2nd Most Common Crime,3rd Most Common Crime,4th Most Common Crime,5th Most Common Crime
1,1957 San Jose Ave,Larceny - From Vehicle,Motor Vehicle Theft,Vandalism,Missing Person,Weapons Offense
3,1501 Greenwich St #202,Larceny - From Vehicle,Vandalism,Larceny Theft - Other,Motor Vehicle Theft,Burglary - Other
4,69 Palm Ave #4,Larceny - From Vehicle,Larceny Theft - Other,Vandalism,Theft From Vehicle,Robbery - Commercial
6,8 Octavia St #301,Larceny - From Vehicle,Vandalism,Larceny Theft - Other,Simple Assault,Motor Vehicle Theft
11,2002 3rd #117,Larceny - From Vehicle,Larceny Theft - Other,Simple Assault,Larceny Theft - From Building,Warrant
16,1139 Judah St,Larceny - From Vehicle,Larceny Theft - Other,Vandalism,Burglary - Residential,Theft From Vehicle
17,1635 29th Ave,Larceny - From Vehicle,Larceny Theft - Other,Larceny Theft - From Building,Vandalism,Theft From Vehicle
19,5090 Diamond Heights Blvd Unit A,Larceny - From Vehicle,Fraud,Larceny Theft - Other,Burglary - Commercial,Vandalism
28,1819 30th Ave,Larceny - From Vehicle,Aggravated Assault,Simple Assault,Larceny Theft - Shoplifting,Warrant
29,1388 Gough St #1101,Larceny - From Vehicle,Larceny Theft - Other,Vandalism,Simple Assault,Motor Vehicle Theft


### Cluster 1

In [22]:
sf_merged.loc[sf_merged['Cluster Labels'] == 1, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Crime,2nd Most Common Crime,3rd Most Common Crime,4th Most Common Crime,5th Most Common Crime
2,1328-1330 Silver Ave,Motor Vehicle Theft,Larceny Theft - Other,Larceny - From Vehicle,Vandalism,Robbery - Other
7,400 Beale St #1012,Larceny Theft - Other,Vandalism,Larceny - From Vehicle,Burglary - Other,Simple Assault
8,222 Parnassus Ave Unit B,Drug Violation,Larceny Theft - Shoplifting,Larceny - From Vehicle,Burglary - Hot Prowl,Burglary - Other
9,1133 Russia Ave,Larceny Theft - Other,Larceny - From Vehicle,Simple Assault,Robbery - Other,Missing Adult
12,99 Rausch St #113,Drug Violation,Larceny - From Vehicle,Other,Simple Assault,Warrant
14,25 Joice St,Larceny Theft - Shoplifting,Larceny - From Vehicle,Larceny Theft - Other,Vandalism,Simple Assault
15,1326 Palou Ave,Vandalism,Larceny - From Vehicle,Larceny Theft - Other,Other,Aggravated Assault
18,520 6th St #15,Larceny - From Vehicle,Warrant,Recovered Vehicle,Vandalism,Simple Assault
21,1879 19th Ave,Vandalism,Larceny - From Vehicle,Other,Stolen Property,Larceny Theft - Other
22,3 Mabrey Ct,Vandalism,Other,Motor Vehicle Theft,Larceny - From Vehicle,Larceny Theft - Other


### Cluster 2

In [23]:
sf_merged.loc[sf_merged['Cluster Labels'] == 2, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Crime,2nd Most Common Crime,3rd Most Common Crime,4th Most Common Crime,5th Most Common Crime
5,878 47th Ave,Larceny - From Vehicle,Vandalism,Larceny Theft - Other,Simple Assault,Motor Vehicle Theft
44,1001 17th St #403,Larceny - From Vehicle,Vandalism,Larceny Theft - Other,Burglary - Other,Warrant


### Cluster 3

In [24]:
sf_merged.loc[sf_merged['Cluster Labels'] == 3, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Crime,2nd Most Common Crime,3rd Most Common Crime,4th Most Common Crime,5th Most Common Crime


### Cluster 4

In [25]:
sf_merged.loc[sf_merged['Cluster Labels'] == 4, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Crime,2nd Most Common Crime,3rd Most Common Crime,4th Most Common Crime,5th Most Common Crime
0,257 N Lake Merced Hls,Simple Assault,Vandalism,Weapons Offense,Fire Report,Larceny Theft - Pickpocket


### Cluster 5

In [26]:
sf_merged.loc[sf_merged['Cluster Labels'] == 5, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Crime,2nd Most Common Crime,3rd Most Common Crime,4th Most Common Crime,5th Most Common Crime
10,663 Lakeview Ave,Vandalism,Motor Vehicle Theft,Aggravated Assault,Larceny - From Vehicle,Simple Assault
13,2166 47th Ave,Motor Vehicle Theft,Vandalism,Missing Person,Larceny - From Vehicle,Recovered Vehicle
20,2078 45th Ave,Motor Vehicle Theft,Larceny - From Vehicle,Recovered Vehicle,Weapons Offense,Forgery And Counterfeiting
27,142 Elmira St,Motor Vehicle Theft,Larceny - From Vehicle,Larceny Theft - Other,Recovered Vehicle,Vandalism
46,1667 Silver Ave,Motor Vehicle Theft,Larceny - From Vehicle,Vandalism,Larceny Theft - Other,Missing Person
